#### Imports

In [106]:
import pandas as pd
import numpy as np

#### Read linkedin csv

In [107]:
linkedin = pd.read_csv('../data/linkedin.csv')
linkedin.sample(3)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
85,3761816473,Data Analyst,Sedgwick,"· Nevada, Estados Unidos",NaN,En remoto,Jornada completa,Sin experiencia,Más de 10.001 empleados · Seguros
138,3766802812,Data Analyst III,WinMax,"· Cupertino, CA",60 $/h - 63 $/h,Presencial,Contrato por obra,Algo de responsabilidad,De 51 a 200 empleados · Dotación y selección d...
3,3577492525,Principal Deep Learning Engineer - Computer Vi...,Roblox,"· San Mateo, CA",283.780 $/año - 331.640 $/año,Híbrido,Jornada completa,Sin experiencia,De 1.001 a 5.000 empleados · Desarrollo de sof...


#### Cleaning 'company_state' column

In [108]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'NC', 'SC', 'CO', 'CT', 'ND', 'SD', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
          'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NJ', 'NY', 'NH', 'NM', 'OH', 'OK', 'OR', 'PA', 'RI', 'TN', 'TX', 'UT',
          'UT', 'VT', 'VA', 'WV', 'WA', 'WI', 'WY']
linkedin['company_state'] = linkedin['company_state'].apply(lambda row: 'Estados Unidos' if row == '· Estados Unidos' else row)
linkedin['company_state'].value_counts()

company_state
Estados Unidos                             33
· Nueva York, NY                           11
· Seattle, WA                               8
· Washington, DC                            7
· Cincinnati, OH                            6
                                           ..
· Newport News, VA                          1
· Morris Plains, NJ                         1
· Washington DC-Baltimore y alrededores     1
· Chesterfield, VA                          1
· Santa Ana, CA                             1
Name: count, Length: 151, dtype: int64

In [109]:
def fclean_company_state(df, states):
    for i in states:
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: i if i in str(row) else row)
    return df

linkedin = fclean_company_state(linkedin, states)

In [110]:
def sclean_company_state(df, states):
    for key, value in states.items():
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: value if key in str(row) else row)
    return df

left_states = {
    'Virginia': 'VA',
    'Minnesota': 'MN',
    'Washington': 'WA',
    'San Francisco': 'CA',
    'Nebraska': 'NE',
    'California': 'CA',
    'Texas': 'TX',
    'Nueva York': 'NY',
    'Michigan': 'MI',
    'Luisiana': 'LA',
    'Florida': 'FL',
    'Maryland': 'MD',
    'Nevada': 'NV',
    'Oregón': 'OR',
    'Oklahoma': 'OK',
    'Georgia': 'GA',
    'Carolina del Sur': 'SC',
    'Delaware': 'DE',
    'Ohio': 'OH',
    'Arkansas': 'AR',
    'Iowa': 'IA',
    'Carolina del Norte': 'NC',
    'Indiana': 'IN',
    'Nashville': 'TN',
    'Massachusetts': 'MA',
    'Tennessee': 'TN',
    'Enid': 'OK'
}
linkedin = sclean_company_state(linkedin, left_states)

#### Cleaning 'company_state' column

In [111]:
linkedin.sample(5)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
210,3772904446,"Data Research Analyst, gt.school (Remote) - $6...",Crossover,NC,30 $/hr,En remoto,Jornada completa,Algo de responsabilidad,De 5.001 a 10.000 empleados · Servicios y cons...
252,3773899227,Data Analyst I,Centene Corporation,Estados Unidos,NaN,En remoto,Jornada completa,NaN,Más de 10.001 empleados · Hospitales y atenció...
194,3771908283,Head of Data (remote),Verve Group,NY,275.000 $/año - 425.000 $/año,En remoto,Jornada completa,Director,De 201 a 500 empleados · Servicios de publicidad
9,3656659700,Deep Learning Research Engineer,Netradyne,CA,145.000 $/año - 180.000 $/año,Híbrido,Jornada completa,Sin experiencia,De 501 a 1.000 empleados · Servicios y consult...
267,3775178848,RESEARCH SCIENTIST 2,University of Washington,WA,4.649 $/mes - 9.465 $/mes,Presencial,Jornada completa,Intermedio,Más de 10.001 empleados · Enseñanza superior


In [112]:
linkedin['remote_ratio'].value_counts()

remote_ratio
Presencial                                                                              107
En remoto                                                                               106
Híbrido                                                                                  38
Jornada completa                                                                          6
162.000 $/año - 247.500 $/año                                                             4
116.002 $/año - 168.000 $/año                                                             3
65.800 $/año - 165.600 $/año                                                              2
145.000 $/año - 355.000 $/año                                                             1
180.000 $/año - 275.000 $/año                                                             1
Contrato por obra                                                                         1
En remotoCoincide con tus preferencias de empleo. La modalidad labo

Aquí podemos ver que hay valores en la columna que pertenecen a la columna salary_range, por lo tanto tienen que moverse a la columna correspondiente.

In [113]:
linkedin['salary_range'] = linkedin['remote_ratio'].apply(lambda row: str(row) if '$' in str(row) else np.nan)
linkedin['remote_ratio'] = linkedin['remote_ratio'].apply(lambda row: np.nan if '$' in str(row) else str(row))
linkedin['remote_ratio'].value_counts()

remote_ratio
Presencial                                                                              107
En remoto                                                                               106
Híbrido                                                                                  38
nan                                                                                      25
Jornada completa                                                                          6
Contrato por obra                                                                         1
En remotoCoincide con tus preferencias de empleo. La modalidad laboral es En remoto.      1
Name: count, dtype: int64

También tenemos valores que pertencen a la columna employment_type, tienen que moverse a la columna correspondiente, pero primero comprobaremos que no sean nulos los valores que hay en la columna employment_type.

In [114]:
linkedin[(linkedin['remote_ratio'] == 'Jornada completa') | (linkedin['remote_ratio'] == 'Contrato por obra')]

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
25,3731939310,Data Scientist,Yext,NY,NaN,Jornada completa,Sin experiencia,NaN,"De 1.001 a 5.000 empleados · Tecnología, infor..."
39,3747569772,Data Engineer,Massachusetts Institute of Technology,MA,NaN,Jornada completa,Sin experiencia,NaN,De 5.001 a 10.000 empleados · Enseñanza superior
50,3752319824,Data Engineer,Metric5,MO,NaN,Jornada completa,Sin experiencia,NaN,De 51 a 200 empleados · Servicios y consultorí...
100,3762746288,Senior Data Engineer,Agile Global Solutions LLC,VA,NaN,Contrato por obra,Intermedio,NaN,Entre 1 y 10 empleados · Servicios y consultor...
142,3767189280,Data Engineer,"GEI Consultants, Inc.",NY,NaN,Jornada completa,Sin experiencia,NaN,De 1.001 a 5.000 empleados · Ingeniería civil
164,3768901276,Data Engineer,Deseret First Credit Union,UT,NaN,Jornada completa,Sin experiencia,NaN,De 51 a 200 empleados · Banca
278,3775610628,Data Engineer,Accenture,FL,NaN,Jornada completa,Intermedio,NaN,Más de 10.001 empleados · Consultoría y servic...


Como hay valores que parece que corresponden a la experiencia vamos a mover estos valores a su columna correspondiente pero estandarizando primero los valores válidos de la columna remote_ratio.

In [115]:
def clean_remote_ratio(df, dict_):
    for key, val in dict_.items():
        df['remote_ratio'] = df['remote_ratio'].apply(lambda row: val if key in str(row) else str(row))
    return df

remote_dict = {
    'Presencial': 0,
    'Híbrido': 50,
    'En remoto': 100
}
linkedin = clean_remote_ratio(linkedin, remote_dict)
linkedin['remote_ratio'].value_counts()

remote_ratio
0                    107
100                  107
50                    38
nan                   36
Jornada completa       6
Contrato por obra      1
Name: count, dtype: int64

In [116]:
def get_experience_level(row):
    if row == 'Sin experiencia' or row == 'Intermedio':
        return row
    else:
        return linkedin.loc[linkedin['employment_type'] == row, 'experience_level'].values[0]
    
def update_employment_type(row):
    if row != 0 and row != 50 and row != 100:
        return row
    else:
        return linkedin.loc[linkedin['remote_ratio'] == row, 'employment_type'].values[0]

linkedin['experience_level'] = linkedin['employment_type'].apply(get_experience_level)
linkedin['employment_type'] = linkedin['remote_ratio'].apply(update_employment_type)
linkedin['remote_ratio'] = linkedin['remote_ratio'].apply(lambda row: np.nan if row == 'Jornada completa' or row == 'Contrato por obra' else row)

The next line should not return any row, that is because the cleaning is working fine.

In [117]:
linkedin[(linkedin['remote_ratio'] == 'Jornada completa') | (linkedin['remote_ratio'] == 'Contrato por obra')]

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size


In [118]:
linkedin['remote_ratio'].value_counts()

remote_ratio
0      107
100    107
50      38
nan     36
Name: count, dtype: int64

#### Cleaning 'company_state' column

In [119]:
linkedin.sample(5)
linkedin['experience_level'].unique()

array(['Intermedio', 'Jornada completa', 'Sin experiencia', 'Prácticas',
       'Algo de responsabilidad'], dtype=object)